# Lesson: Differential Privacy for Deep Learning

So in the last lessons you may have been wondering - what does all of this have to do with Deep Learning? Well, these same techniques we were just studying form the core primitives for how Differential Privacy provides guarantees in the context of Deep Learning. 

Previously, we defined perfect privacy as "a query to a database returns the same value even if we remove any person from the database", and used this intuition in the description of epsilon/delta. In the context of deep learning we have a similar standard.

Training a model on a dataset should return the same model even if we remove any person from the dataset.

Thus, we've replaced "querying a database" with "training a model on a dataset". In essence, the training process is a kind of query. However, one should note that this adds two points of complexity which database queries did not have:

    1. do we always know where "people" are referenced in the dataset?
    2. neural models rarely never train to the same output model, even on identical data

The answer to (1) is to treat each training example as a single, separate person. Strictly speaking, this is often overly zealous as some training examples have no relevance to people and others may have multiple/partial (consider an image with multiple people contained within it). Thus, localizing exactly where "people" are referenced, and thus how much your model would change if people were removed, is challenging.

The answer to (2) is also an open problem - but several interesitng proposals have been made. We're going to focus on one of the most popular proposals, PATE.

## An Example Scenario: A Health Neural Network

First we're going to consider a scenario - you work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier. 

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which DO have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

- 1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
- 2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
- 3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
- 4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.

So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [0]:
import numpy as np

In [0]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [0]:
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0) # fake predictions

In [0]:
new_labels = list()
for an_image in preds:

    label_counts = np.bincount(an_image, minlength=num_labels)

    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [6]:
new_labels


[7,
 1,
 8,
 6,
 8,
 3,
 9,
 0,
 7,
 0,
 5,
 2,
 2,
 8,
 2,
 0,
 0,
 1,
 7,
 4,
 7,
 8,
 4,
 0,
 9,
 4,
 1,
 0,
 4,
 0,
 3,
 2,
 7,
 7,
 6,
 8,
 3,
 1,
 8,
 8,
 5,
 7,
 8,
 8,
 2,
 1,
 5,
 0,
 0,
 6,
 4,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 6,
 5,
 0,
 8,
 9,
 9,
 1,
 6,
 9,
 9,
 1,
 6,
 2,
 8,
 2,
 9,
 9,
 5,
 2,
 5,
 7,
 9,
 6,
 8,
 8,
 5,
 9,
 1,
 1,
 0,
 4,
 6,
 9,
 0,
 8,
 8,
 0,
 3,
 5,
 9,
 1,
 7,
 0,
 9,
 0,
 3,
 3,
 5,
 4,
 5,
 4,
 3,
 6,
 5,
 7,
 5,
 7,
 7,
 8,
 3,
 4,
 5,
 5,
 1,
 7,
 2,
 5,
 6,
 3,
 4,
 3,
 1,
 3,
 6,
 3,
 1,
 8,
 6,
 2,
 4,
 4,
 7,
 0,
 4,
 5,
 7,
 4,
 5,
 8,
 2,
 7,
 0,
 7,
 1,
 7,
 4,
 8,
 2,
 1,
 3,
 0,
 4,
 2,
 7,
 3,
 9,
 2,
 5,
 6,
 3,
 8,
 1,
 2,
 4,
 3,
 1,
 4,
 6,
 9,
 6,
 8,
 3,
 2,
 4,
 2,
 1,
 2,
 3,
 5,
 1,
 1,
 6,
 7,
 1,
 7,
 1,
 7,
 2,
 5,
 4,
 2,
 1,
 4,
 8,
 2,
 5,
 7,
 6,
 0,
 1,
 2,
 2,
 6,
 2,
 7,
 5,
 1,
 1,
 9,
 8,
 8,
 3,
 8,
 4,
 0,
 4,
 9,
 5,
 7,
 4,
 1,
 7,
 8,
 6,
 6,
 1,
 4,
 0,
 1,
 5,
 2,
 2,
 6,
 4,
 0,
 3,
 2,
 6,
 0,
 4,
 2,
 1,
